### Programming Assignment 1 Q1
### Decision Tree

Group Number : 46

Student 1 : Aryan Singh 19CS30007

Student 2 : Seemant Guruprasad Achari 19CS30057


In [92]:
import numpy as np
import pandas as pd
from graphviz import Digraph

In [93]:
# A constant to reference the result column
LABEL = 'Response'

# Data read using pandas
df = pd.read_csv('Dataset_C.csv')

In [94]:
# splitting dataset index
def train_test_split(df,train_sample=0.5,target_col= LABEL):
    all_indexes=[]
    df_grouped= df.groupby(target_col)
    for x,x_df in df_grouped:
        t = x_df.sample(frac=train_sample,).index
        all_indexes.append(t)
    g = all_indexes[0].values
    for k in all_indexes[1:]:
        g=np.hstack([g,k.values])
        #np.hstack([all_indexes[0].values,all_indexes[1].values])

    train_df = df[df.index.isin(g) ]
    test_df = df[~df.index.isin(g)]
    return train_df, test_df 

In [95]:
train_set, test_set = train_test_split(df, .8)

In [96]:
print(train_set[LABEL].value_counts())
print(test_set[LABEL].value_counts())


0    92454
1    12898
Name: Response, dtype: int64
0    23113
1     3224
Name: Response, dtype: int64


In [97]:
# Checking the Top five rows of the Data
train_set.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1,2,Male,76,1,3,0,1-2 Year,No,33536,26,183,0
3,4,Male,21,1,11,1,< 1 Year,No,28619,152,203,0
4,5,Female,29,1,41,1,< 1 Year,No,27496,152,39,0
5,6,Female,24,1,33,0,< 1 Year,Yes,2630,160,176,0
6,7,Male,23,1,11,0,< 1 Year,Yes,23367,152,249,0


In [225]:
# Finding the Discrete Valued attributes in the dataset

# max unique categories to consider
MAX_UNIQUE_CATEGORIES = 20
for k in df.columns:
    if k == 'id':
        continue
    if k == LABEL :
        print("Label     => ", end = "")
    else:
        print("Attribute => ", end = "")
    print(f"{k} has {len(df[k].unique())} unique values.", end="")
    if(len(df[k].unique()) <= MAX_UNIQUE_CATEGORIES):
        print("[ ", end = "")
        for val in df[k].unique():
            print(val, end = ", ")
        print("]", end="")
    print()

Attribute => Gender has 2 unique values.[ Male, Female, ]
Attribute => Age has 66 unique values.
Attribute => Driving_License has 2 unique values.[ 1, 0, ]
Attribute => Region_Code has 53 unique values.
Attribute => Previously_Insured has 2 unique values.[ 0, 1, ]
Attribute => Vehicle_Age has 3 unique values.[ > 2 Years, 1-2 Year, < 1 Year, ]
Attribute => Vehicle_Damage has 2 unique values.[ Yes, No, ]
Attribute => Annual_Premium has 36617 unique values.
Attribute => Policy_Sales_Channel has 140 unique values.
Attribute => Vintage has 290 unique values.
Label     => Response has 2 unique values.[ 1, 0, ]


In [226]:
CATEGORICAL_FEATURE = "categorical"
CONTINUOUS_FEATURE = "continuous"
def determineTypeOfFeature(df):
    
    feature_types = {}
    n_unique_values_treshold = MAX_UNIQUE_CATEGORIES
    for feature in df.columns:
        if feature == 'id':
            continue
        if feature != LABEL:
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types[feature] = CATEGORICAL_FEATURE
            else:
                feature_types[feature] = CONTINUOUS_FEATURE
    
    return feature_types
FEATURE_TYPES = determineTypeOfFeature(df)
for k, v in FEATURE_TYPES.items():
    print(f"Attr {k} is {v}")

Attr Gender is categorical
Attr Age is continuous
Attr Driving_License is categorical
Attr Region_Code is continuous
Attr Previously_Insured is categorical
Attr Vehicle_Age is categorical
Attr Vehicle_Damage is categorical
Attr Annual_Premium is continuous
Attr Policy_Sales_Channel is continuous
Attr Vintage is continuous


In [227]:
POTENTIAL_SPLIT_LIMIT = 10
def getPotentialSplits(featureName, trainData):
    
    potential_splits = []
    values = trainData[featureName]
    unique_values = np.unique(values)
    
    type_of_feature = FEATURE_TYPES[featureName]
    if type_of_feature == CONTINUOUS_FEATURE:
        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits.append(potential_split)
        
        # Since the potential_splits can have thousands of options, we need to drop some of them to reduce the complexity
        if(len(potential_splits) > POTENTIAL_SPLIT_LIMIT):
            # startIdx = 0
            endIdx = len(potential_splits)
            incrementBy = (endIdx + POTENTIAL_SPLIT_LIMIT -1)// POTENTIAL_SPLIT_LIMIT
            # reducedPotentialList = [potential_splits[idx] for idx in range(startIdx, endIdx, incrementBy)]
            potential_splits = potential_splits[::incrementBy]
    # feature is categorical
    # (there need to be at least 2 unique values, otherwise in the
    # split_data function data_below would contain all data points
    # and data_above would be empty)
    else:
        potential_splits = unique_values
    
    return potential_splits

In [228]:
getPotentialSplits('Vintage', df)

[10.5, 39.5, 68.5, 97.5, 126.5, 155.5, 184.5, 213.5, 242.5, 271.5]

In [229]:
def getFeaturesList(trainData, featureSet):
    featuresList = []
    for key, value in FEATURE_TYPES.items():
        if key not in featureSet:
            continue
        if(value == CONTINUOUS_FEATURE):
            potentialSplits = getPotentialSplits(key, trainData)
            for potentialSplit in potentialSplits:
                featuresList.append([key, potentialSplit])
        else:
            featuresList.append([key])
    
    return featuresList  


In [230]:
# Method to Calulate Total Entropy

# Input : Traning Data, label, classList

def calculateTotalEntropy(trainData, label, classList):
    totalCount = trainData.shape[0]

    totalEntropy = 0

    for c in classList:
        # Count the number of data points which satify == c class constraints
        totalClassCount = trainData[trainData[label] == c].shape[0]
        # Calulate the class's entropy
        totalClassEntropy = 0
        prob = totalClassCount / totalCount
        if( prob >0):
            totalClassEntropy = - (prob)*np.log2(prob)
        # Accumulate the class's entropy into the totalEntropy
        totalEntropy = totalEntropy + totalClassEntropy
    
    return totalEntropy

def calculateEntropy(featureValueData, label, classList):
    classCount = featureValueData.shape[0]
    entropy = 0

    for c in classList:
        # Count the number of data points withing the featureValueData which satify == c class constraints
        labelClassCount = featureValueData[featureValueData[label] == c].shape[0]
        classEntropy = 0
        if labelClassCount != 0:
            # if labelClassCount != 0 then calculate its entropy
            prob = labelClassCount/classCount
            if prob !=0:
                classEntropy = -prob * np.log2(prob)
        # Accumulate its entropy
        entropy = entropy + classEntropy
    return entropy

def calulateInformationGain(feature, trainData, label, classList):
    featureName = feature[0]
    threshold = None

    # featureValueList = getPotentialSplits(featureName, trainData)
    # featureValueList = trainData[featureName].unique() # here UPDATE

    totalCount = trainData.shape[0]
    featureInfo = 0.0
        
    if(FEATURE_TYPES[featureName] == CONTINUOUS_FEATURE):
        # For cases when value <= threshold
        threshold = feature[1]
        featureValueData = trainData[trainData[featureName] <= threshold]
        featureValueCount = featureValueData.shape[0]
        featureValueEntropy = calculateEntropy(featureValueData, label, classList)
        featureValueProbability = featureValueCount / totalCount
        featureInfo += featureValueProbability * featureValueEntropy

        # For cases when value > threshold
        featureValueData = trainData[trainData[featureName] > threshold]
        featureValueCount = featureValueData.shape[0]
        featureValueEntropy = calculateEntropy(featureValueData, label, classList)
        featureValueProbability = featureValueCount / totalCount
        featureInfo += featureValueProbability * featureValueEntropy
    else:
        featureValueList = trainData[featureName].unique()
        for featureValue in featureValueList:
            featureValueData = trainData[trainData[featureName] == featureValue]
            featureValueCount = featureValueData.shape[0]
            featureValueEntropy = calculateEntropy(featureValueData, label, classList)
            featureValueProbability = featureValueCount / totalCount
            featureInfo += featureValueProbability * featureValueEntropy
    
    totalEntropy = calculateTotalEntropy(trainData, label, classList)

    gain = totalEntropy - featureInfo
    
    return gain 

def findMostImportantFeature(trainData, featureList, label, classList):
    # featureList = trainData.columns.drop(label) # Since label is not a feature
    # featureList = getFeaturesList(trainData) 
    
    maxInfoGain = -1 # init with smallest value possible
    maxInfoFeature = None
    # features are of two types: 
    # 1) Categorical, in which you iterate over multiple value of that feature
    # 2) Continous converted into binary category, i/p<= threshold or ip>threshold
    for feature in featureList:
        featureInfoGain = calulateInformationGain(feature, trainData, label, classList)
        # print(feature, featureInfoGain, maxInfoGain)
        if float(featureInfoGain) > float(maxInfoGain):
            maxInfoGain = featureInfoGain
            maxInfoFeature = feature
    return maxInfoFeature

In [231]:
print(f"The entropy of Complete Data Set with {LABEL} as label = {calculateTotalEntropy(df, LABEL, df[LABEL].unique())}")

The entropy of Complete Data Set with Response as label = 0.5362909567366381


In [232]:
calulateInformationGain(['Previously_Insured',298.5], df, LABEL, df[LABEL].unique())


0.1147977301372255

In [233]:
calulateInformationGain(['Vintage',150.5], df, LABEL, df[LABEL].unique())

1.4327494857191425e-07

In [234]:
calulateInformationGain(['Age',2900], df, LABEL, df[LABEL].unique())

0.0

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131689 entries, 0 to 131688
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    131689 non-null  int64 
 1   Gender                131689 non-null  object
 2   Age                   131689 non-null  int64 
 3   Driving_License       131689 non-null  int64 
 4   Region_Code           131689 non-null  int64 
 5   Previously_Insured    131689 non-null  int64 
 6   Vehicle_Age           131689 non-null  object
 7   Vehicle_Damage        131689 non-null  object
 8   Annual_Premium        131689 non-null  int64 
 9   Policy_Sales_Channel  131689 non-null  int64 
 10  Vintage               131689 non-null  int64 
 11  Response              131689 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 12.1+ MB


In [236]:
FEATURE_TYPES.items()

dict_items([('Gender', 'categorical'), ('Age', 'continuous'), ('Driving_License', 'categorical'), ('Region_Code', 'continuous'), ('Previously_Insured', 'categorical'), ('Vehicle_Age', 'categorical'), ('Vehicle_Damage', 'categorical'), ('Annual_Premium', 'continuous'), ('Policy_Sales_Channel', 'continuous'), ('Vintage', 'continuous')])

Class for decision node and tree

In [237]:
#  passing same feature list

class DecisionNode:
    nodeId = 0
    def __init__(self, featureSet, label, classList, decisionNodeName, depth, max_depth):
        # print(depth, decisionNodeName)
        self.nodeId = DecisionNode.nodeId
        DecisionNode.nodeId += 1
        self.featureSet = featureSet
        # print(f"Cur feature set{self.featureSet}")
        self.decisionNodeName = decisionNodeName
        self.label = label
        self.classList = classList
        self.leafNode = False
        self.isPruned = False
        self.depth = depth
        self.max_depth = max_depth
        if( self.depth == max_depth ):
            self.isPruned = True
    
    def fitData(self, trainData):
        label = self.label
        classList = self.classList
        self.featureList = getFeaturesList(trainData, self.featureSet)
        self.labelDistribution = {}
        for c in self.classList:
            cnt = trainData[trainData[label]==c].shape[0]
            self.labelDistribution[c] = cnt
        # print(self.labelDistribution )
        self.trainDataCount = trainData.shape[0]
        self.currentNodeReturnVal = trainData[label].mode()[0]  # return this if you cant decide
        self.inpurity = calculateTotalEntropy(trainData, label, classList)
        # print(f"Impurity = {self.inpurity} @depth = {self.depth}")
        if(self.labelDistribution[0]==0 or self.labelDistribution[1] == 0 or self.isPruned):
            self.leafNode = True

        self.childrenNode = {}
        if(not self.leafNode):
            label = self.label
            classList = self.classList
            retVal = findMostImportantFeature(trainData, self.featureList, label, classList)
            if(retVal == None):
                self.leafNode = True
                self.decisionNodeName = self.decisionNodeName
                self.bestFeatureName = "Ans = " + str(self.currentNodeReturnVal)
                return
            self.bestFeatureName = retVal[0]
            # print(self.bestFeatureName)
            newFeatureSet = self.featureSet.copy()
            # newFeatureSet.remove(self.bestFeatureName)
            if(FEATURE_TYPES[self.bestFeatureName] == CONTINUOUS_FEATURE):
                self.threshold = retVal[1]
                # print(f"threshold = {self.threshold}")
                # left is <= threshold
                nextNodeName = self.bestFeatureName + " <= " + str(self.threshold)
                # print(f"The next node will go by the name {nextNodeName}")
                node = DecisionNode(newFeatureSet, self.label, self.classList, nextNodeName, self.depth + 1, self.max_depth)
                node.fitData(trainData[trainData[self.bestFeatureName] <= self.threshold])
                self.childrenNode[True] = (node)

                # right is > threshold
                nextNodeName = self.bestFeatureName + " > " + str(self.threshold)
                # print(f"The next node will go by the name {nextNodeName}")
                node = DecisionNode(newFeatureSet, self.label, self.classList, nextNodeName, self.depth + 1, self.max_depth)
                node.fitData(trainData[trainData[self.bestFeatureName] > self.threshold])
                self.childrenNode[False] = (node)

            else:
                attributeValues = trainData[self.bestFeatureName].unique()
                for nextValue in attributeValues:
                    nextNodeName = self.bestFeatureName + " == " + str(nextValue)
                    # print(f"The next node will go by the name {nextNodeName}")
                    node = DecisionNode(newFeatureSet, self.label, self.classList, nextNodeName, self.depth + 1, self.max_depth)
                    node.fitData(trainData[trainData[self.bestFeatureName] == nextValue])
                    # print(node)
                    self.childrenNode[nextValue] = node

        else:
            
            self.decisionNodeName = self.decisionNodeName
            self.bestFeatureName = "Ans = " + str(self.currentNodeReturnVal)
    def predict(self, x_hat):
        if(self.leafNode or self.isPruned):
            # returning the majority decision
            return self.currentNodeReturnVal
        # we need to traverse down the tree
        if(FEATURE_TYPES[self.bestFeatureName] == CONTINUOUS_FEATURE):
            nextNode = (x_hat[self.bestFeatureName] <= self.threshold)
        else:
            nextNode = x_hat[self.bestFeatureName]
        if(nextNode not in self.childrenNode.keys()):
            # print(self.bestFeatureName)
            # print(f"xhat = {x_hat}")
            # print(f"nextNode = {nextNode}")
            # print(f"childrenNode = {self.childrenNode.keys()}")
            return self.currentNodeReturnVal
        return self.childrenNode[nextNode].predict(x_hat)


    def __repr__(self):
        return f"Name: {self.decisionNodeName}\nData Count: {self.trainDataCount}\nFeature: {self.bestFeatureName}\nChildren: {len(self.childrenNode)}\n"


        

In [238]:
class DecisionTree:
    def __init__(self, featureSet, label, classList, rootName, max_depth):
        DecisionNode.nodeId = 0
        self.max_depth = max_depth
        self.root= DecisionNode(featureSet, LABEL, classList, rootName, 0, max_depth)
        self.treeDepth = 0
    def fitData(self, trainData):
        self.root.fitData(trainData)
    def predict(self, x):
        return self.root.predict(x)
    def getTreeDepth(self):
        maxDepthTillNow = 0
        node = self.root
        st = [node]

        while(len(st)>0):
            node = st.pop()
            maxDepthTillNow = max(maxDepthTillNow, node.depth)
            if(node.leafNode):
                continue
            for key in node.childrenNode:
                st.append(node.childrenNode[key])
        return maxDepthTillNow

In [112]:
# featureList = getFeaturesList(train_set)
featureSet = set(FEATURE_TYPES.keys())
tree= DecisionTree(featureSet, LABEL, train_set[LABEL].unique(), "Root", 10)


In [113]:
tree.fitData(train_set)

In [114]:
# cnt = 0
# outputs = {
#     0: 0,
#     1: 0
# }
# total_samples = 0
# for index, row in test_set.iterrows():
#     y_hat = root.predict(row)
#     outputs[y_hat] += 1
#     cnt +=  (y_hat == row[LABEL])
#     total_samples +=1
#     # print(f"pediction = {root.predict(row)}, Real ={row[LABEL]}")
# print(cnt)
# print(outputs)
# print(f"Acc = {cnt / total_samples}")
# print(self.bestFeatureName, x_hat, nextNode, self.childrenNode.keys())

def GetAccuracy(tree:DecisionTree, test_set, printDetails = True):
    cnt = 0
    outputs = {
        0: 0,
        1: 0
    }
    total_samples = 0
    root = tree.root
    for index, row in test_set.iterrows():
        y_hat = root.predict(row)
        outputs[y_hat] += 1
        cnt +=  (y_hat == row[LABEL])
        total_samples +=1
        # print(f"pediction = {root.predict(row)}, Real ={row[LABEL]}")
    accuracy = cnt / total_samples
    if(printDetails):
        print(cnt)
        print(outputs)
        print(f"Accuracy = {cnt / total_samples} and Depth = {tree.getTreeDepth()}")
    
    return accuracy

            

In [115]:
GetAccuracy(tree, test_set)

23068
{0: 26232, 1: 105}
Accuracy = 0.8758780422979079 and Depth = 10


0.8758780422979079

In [116]:
test_set[LABEL].value_counts()

0    23113
1     3224
Name: Response, dtype: int64

In [117]:
def getDetail(node:DecisionNode):
    return f"Node id {node.nodeId}\n {node.bestFeatureName}"
def print_tree(tree:DecisionTree, op_file):
        '''
            This function prints the decision tree using the graphviz library.
            It basically does a bfs on the graph and then appends the nodes one by one.
        '''
        root = tree.root
        my_graph= Digraph('Decision Tree', filename=op_file)
        my_graph.attr(rankdir='TD', size='1000,500')

        my_graph.attr('node', shape='rectangle')
        
        # doing a bfs using a queue
        qq = [root]                          # using a list as a queue for the bradth first search
        while len(qq) > 0:
            node = qq.pop(0)         
            if node.leafNode:                    # stop if its a leaf node
                continue
            for _,child in node.childrenNode.items():    # else check its children
                my_graph.edge(getDetail(node), getDetail(child), label=child.decisionNodeName)
                qq.append(child)

        my_graph.render(op_file, view=True)     # open the output file for convenience

        return 

In [118]:
print_tree(tree, 'pic.gv')

In [119]:
trees = []
for i in range(10):
    # train_set, test_set = train_test_split(df, .8)
    splitRatio = np.random.uniform(0.5, .8)
    print(f"Testing #{i} with splitRatio = {splitRatio}")
    train_set, test_set = train_test_split(df, splitRatio)
    featureSet = set(FEATURE_TYPES.keys())
    tree= DecisionTree(featureSet, LABEL, train_set[LABEL].unique(), f"Root #{i}", 8)
    tree.fitData(train_set)
    print(f"Testing tree #{i}")
    GetAccuracy(tree, test_set)
    trees.append(tree)
    print()

Testing #0 with splitRatio = 0.6746651465734925
Testing tree #0
37599
{0: 42836, 1: 7}
Accuracy = 0.8775996078705973 and Depth = 8

Testing #1 with splitRatio = 0.5736114001678966
Testing tree #1
49256
{0: 56130, 1: 20}
Accuracy = 0.8772217275155832 and Depth = 8

Testing #2 with splitRatio = 0.5410974621211172
Testing tree #2
53010
{0: 60392, 1: 40}
Accuracy = 0.8771842732327244 and Depth = 8

Testing #3 with splitRatio = 0.7492545538547516
Testing tree #3
28941
{0: 32848, 1: 173}
Accuracy = 0.8764422640138094 and Depth = 8

Testing #4 with splitRatio = 0.712126354123391
Testing tree #4


KeyboardInterrupt: 

In [210]:
def pruneTree(tree:DecisionTree, validation_set):
    maxValAcc = GetAccuracy(tree, validation_set, False)
    print(f"Base Acc = {maxValAcc}")
    node = tree.root
    st = [node]

    nodeToPrune = None
    # print("I am here")
    while(len(st)>0):
        node = st.pop()
        # mark node as pruned
        if(node.leafNode):
            continue

        node.isPruned = True
        # evaluate
        curAccuracy = GetAccuracy(tree, validation_set, False)
        # maxValAcc = max(maxValAcc, curAccuracy)

        node.isPruned = False
        # print(curAccuracy, node.nodeId)
        # print(curAccuracy, maxValAcc, curAccuracy> maxValAcc)
        # store node id if better found
        if( curAccuracy > maxValAcc):
            nodeToPrune = node
            print(f"Increased accuracy from {maxValAcc} to {curAccuracy}")
            maxValAcc = curAccuracy

            # remove
            # node.isPruned = True
            # node.leafNode = True
            # return True
            # remove
        # loop
        for key in node.childrenNode:
            st.append(node.childrenNode[key])
    if(nodeToPrune is not None):
        nodeToPrune.isPruned = True
        nodeToPrune.leafNode = True
        print(f"Pruned nodeId = {nodeToPrune.nodeId}")
        return True
    
    return False

In [250]:

train_set, test_set = train_test_split(df.head(1000), .8)
train_set, validation_set = train_test_split(train_set, .8)

In [251]:
featureSet = set(FEATURE_TYPES.keys())
tree= DecisionTree(featureSet, LABEL, train_set[LABEL].unique(), "Root", 10)
tree.fitData(train_set)

In [252]:
print_tree(tree, 'pic_unpruned.gv')

In [253]:
GetAccuracy(tree, validation_set)

138
{0: 138, 1: 22}
Accuracy = 0.8625 and Depth = 10


0.8625

In [254]:
cnt = 0
while(pruneTree(tree, validation_set)):
    print(f"pruned {cnt}")
    cnt+=1
    # break

Base Acc = 0.8625
Increased accuracy from 0.8625 to 0.875
Increased accuracy from 0.875 to 0.8875
Pruned nodeId = 2
pruned 0
Base Acc = 0.8875
Increased accuracy from 0.8875 to 0.89375
Pruned nodeId = 70
pruned 1
Base Acc = 0.89375


In [255]:
print_tree(tree, 'pic_pruned.gv')

In [256]:
GetAccuracy(tree, test_set)

159
{0: 174, 1: 26}
Accuracy = 0.795 and Depth = 10


0.795